# Automation Of Champion Challenger

![Champion Challenger Flow](https://raw.githubusercontent.com/sheena-n/Snowflake-Champion-Challenger/dev/images/champion_challenger_flow.png)


Automation Scenario: 
 1. Retrain the model weekly with the latest data.
 2. New trained model becomes CHALLENGER
 3. Compare performance of CHALLENGER to CHAMPION model.
 4. If CHALLENGER Performs better swap the model and CHALLENGER becomes new CHAMPION.


In [ ]:
# Import python packages
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
from snowflake.core.task import Cron
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)
session = get_active_session()

# Set up the database, schema and model registry
database_name='DEV_AUTOMATION_DEMO'
schema_name='CHAMPION_CHALLENGER'
session.use_database(database_name)
session.use_schema(schema_name) 

# Create the tasks using the DAG API
dag_name = "Champion_Challenger_DAG"
warehouse_name = "COMPUTE_WH" #Use any existing WH or create a new one

api_root = Root(session)
schema = api_root.databases[database_name].schemas[schema_name]
dag_op = DAGOperation(schema)    


# Create Automation of two tasks
1. Retraining Task
2. Model Deployment Task

In [ ]:
# Define the DAG - run every weekly monday 1AM 
with DAG(dag_name, schedule=Cron("1 1 * * 1", "Asia/Singapore"), warehouse=warehouse_name) as dag:

    #Challenger model building
    dag_task1 = DAGTask("WEEKLY_RETRAINING", definition="EXECUTE NOTEBOOK DEV_AUTOMATION_DEMO.CHAMPION_CHALLENGER.CC_3_CHALLENGER_TRAINING()", warehouse=warehouse_name)
    #Evaluate and promote models
    dag_task2 = DAGTask("AUTO_DEPLOY_BEST_MODEL", definition="EXECUTE NOTEBOOK DEV_AUTOMATION_DEMO.CHAMPION_CHALLENGER.CC_4_SWAP_MODELS()", warehouse=warehouse_name)

    # Define the dependencies between the tasks
    dag_task1 >> dag_task2 # dag_task1 is a predecessor of dag_task2

# Create the DAG in Snowflake
dag_op.deploy(dag, mode="orreplace")

In [ ]:
# Run the DAG.
dag_op.run(dag) #You can run the DAG from the UI too under DB>SCHEMA>TASK